### Парсер объявлений директа, парсит только первую страницу

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
minus_list = '-фото'
# Список регионов
# https://tech.yandex.ru/xml/doc/dg/reference/regions-docpage/
lr = 1 #регион
n_l = 0 #количесво запрашиваемых страниц в данном скрипте не активно

In [2]:
# Читаем запросы из файла
def read_query_(file):
    
    f = open(file, mode = 'r', encoding = 'utf-8') 
    
    return [x.strip() for x in f]

# заголовок для Яши
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

# Отправка запроса
def request_(query, minus_list,lr, headers, n = 0):
    
    url = 'https://www.yandex.ru/search/ads?text='\
    + query + ' ' + minus_list + '&lr=' + str(lr) + '&p=' + str(n)
    
    return requests.get(url, headers = headers, stream = True)


def pars_(r):
# Добавим закрывающие теги </li> для удобства парсинга    
    mod_r = r.text.replace('<li class=serp-item','</li><li class=serp-item')
    
# Загрузим в BeautifulSoup и получим список всех объявлений хранимых в <li class=serp-item    
    return BeautifulSoup(mod_r,'html.parser').findAll('li', class_='serp-item')

def deep_pars_(parse_bs, now, query):
#     Заголовок объявления
    title_ads = parse_bs.find(class_='organic__url-text').get_text()
#     Позиция объявления
    position = int(parse_bs.get('data-cid')) + 1
#    Домен объявления
    try:
        domain_ads = parse_bs.find(class_='path path_show-https organic__path').a.text
    except:
        domain_ads = 'no domain'
#    Домен объявления если нет видимой ссылки        
    try:
        domain_ads = parse_bs.find(class_='path organic__path').a.text
    except:
        0
        
#    Отображаемая ссылка  
    try:
        path_ads = parse_bs.find(class_='path path_show-https organic__path').text
    except:
        path_ads = 'no path'
#    Текст объявления
    text_ads = parse_bs.find(class_='text-container typo typo_text_m typo_line_m organic__text').text
#    Текст быстрых ссылок, только 4-е штуки описание быстрых ссылок не парсятся
    fl_path_ads = parse_bs.find(class_='sitelinks sitelinks_size_m organic__sitelinks')
    if not fl_path_ads:
        fl_path_ads_1 = fl_path_ads_2 = fl_path_ads_3 = fl_path_ads_4 = 'NO'
    else:
        fl_path_ads = fl_path_ads.findAll('a')
        try:
            fl_path_ads_1 = fl_path_ads[0].get_text()
        except:
            fl_path_ads_1 = 'no fast link'
        try:
            fl_path_ads_2 = fl_path_ads[1].get_text()
        except:
            fl_path_ads_2 = 'no fast link'
        try:
            fl_path_ads_3 = fl_path_ads[2].get_text()
        except:
            fl_path_ads_3 = 'no fast link'
        try:
            fl_path_ads_4 = fl_path_ads[3].get_text()
        except:
            fl_path_ads_4 = 'no fast link'            
#    Время работы 
    try:
        hours_ads = parse_bs.find(class_='serp-meta__item serp-meta__item_last')
    except:
        hours_ads = ''
#    Телефон
    try:
        contact = parse_bs.find(class_='serp-meta serp-meta_type_black typo typo_text_m typo_line_s')
    except:
        contact = ''
    if not contact:
        phone_ads, hour_work_ads = 'no phone', 'no hour of work'
    else:
        contact = contact.findAll('div')
        try:
            phone_ads = contact[1].get_text()
        except:
            phone_ads = 'no phone'
        try:
            hour_work_ads = contact[2].get_text()
        except:
            hour_work_ads = 'no hour of work'
#    Вернет список из элементов
    return [now, position,query ,domain_ads, title_ads, text_ads, path_ads, 
            fl_path_ads_1, fl_path_ads_2, fl_path_ads_3, 
            fl_path_ads_4, phone_ads, hour_work_ads]

In [3]:
query_list = read_query_('queries.txt')

In [4]:
answer = []
for query in query_list:
    now = datetime.datetime.now().strftime("%d-%m-%Y %H:%M")
    r = request_(query, minus_list,lr, headers, n = 0)
    parse = pars_(r)
    print(query)
    for parse_bs in parse:
        answer.append(deep_pars_(parse_bs, now, query))

купить квартиру
цена квартир


In [5]:
pd.set_option('max_colwidth', 400)
pd.set_option('display.width', 10000)
df = pd.DataFrame(answer, columns = ['ДатаВремя', 'Позиция','Запрос', 'Домен','Заголовок','Текст' ,'Отбр.Ссылка', 'бс1', 'бс2', 'бс3', 'бс4', 'Телефон', 'ЧасыРаботы'])
df['ДатаВремя'] = pd.to_datetime(df['ДатаВремя'])
df['Позиция'] = df['Позиция'].astype('int32')

In [6]:
#df.loc[df['Домен']=='realty.yandex.ru', ['Текст','Позиция']] Так можно фильтровать домены
df

,ДатаВремя,Позиция,Запрос,Домен,Заголовок,Текст,Отбр.Ссылка,бс1,бс2,бс3,бс4,Телефон,ЧасыРаботы
0,2020-06-03 14:45:00,1,купить квартиру,a101.ru,"Купите квартиру в Москве от 4,5 млн – Ипотека от 3,75%!",Скидки до 5%! ЖК рядом с метро. С отделкой и без. Прописка в Москве. Звоните сейчас! · Инфраструктура. Ипотека: год без платежей. м. Новомосковская 700м. · Проектная декларация на рекламируемом сайте. З… Читать ещёСкидки до 5%! ЖК рядом с метро. С отделкой и без. Прописка в Москве. Звоните сейчас! · Инфраструктура. Ипотека: год без платежей. м. Новомосковская 700м. · Проектная декларация на ре...,a101.ru›Новостройки,4 ЖК в НАО,"В ипотеку от 3,75%",Старт продаж,Выбор квартиры,+7 (495) 127-36-41,пн-вс 9:00-21:00
1,2020-06-03 14:45:00,2,купить квартиру,etalonnormand.ru,Купить квартиру в ЖК – рядом с метро Бабушкинская,"От 7,9 млн ₽. Двор без машин. 3 минуты пешком до парка. Рассрочка 0%. Звоните! · Подземный паркинг. Свой выход к реке Яузе. В окружении 4х парков. Свой детский сад. 3-6 апартаментов на этаже · Проектная декларация на рекламируемом сайте. З… Читать ещёОт 7,9 млн ₽. Двор без машин. 3 минуты пешком до парка. Рассрочка 0%. Звоните! · Подземный паркинг. Свой выход к реке Яузе. В окружении 4х парков...",etalonnormand.ru›ЖК-Нормандия,Рассрочка 0%,Ипотека,Trade-in,Контакты,+7 (495) 126-71-89,пн-вс 9:00-21:00
2,2020-06-03 14:45:00,3,купить квартиру,realty.yandex.ru,Купите квартиру на вторичном рынке / realty.yandex.ru,Большой выбор проверенных квартир. Доступна история объявлений! · Поиск по карте. Фильтрация по ценам,realty.yandex.ru›Продажа-квартир,От собственников,С парковкой,В 10 мин от метро,Рядом парк,no phone,no hour of work
3,2020-06-03 14:45:00,4,купить квартиру,avito.ru,Продажа недвижимости – Авито Недвижимость,Продажа и аренда квартир. Актуальные объявления на сайте · Безопасно. Удобно. Быстро · Проектная декларация на рекламируемом сайте,avito.ru,Безопасная сделка,Главная,Авито Недвижимость,Подать объявление,no phone,no hour of work
4,2020-06-03 14:45:00,5,купить квартиру,life.grand-deluxe.ru,Элитные квартиры в ЦАО – С отделкой и без,Элитные квартиры на Плющихе! Лучшая инфраструктура в пешей доступности. Звоните! · Роскошные интерьеры. Панорамные окна. Тихий центр Москвы. Высокие потолки. Белоснежные колонны. Натуральный мрамор. Фитнес-клуб. Прогулочный парк · Проектная декларация на рекламируемом сайте. З… Читать ещёЭлитные квартиры на Плющихе! Лучшая инфраструктура в пешей доступности. Звоните! · Роскошные интерьеры. Пан...,life.grand-deluxe.ru›Жизнь-на-Плющихе,Выбор квартир,Элитный район,Ипотека,Уникальная архитектура,+7 (495) 241-59-35,"пн-пт 9:00-21:00, сб-вс 10:00-20:00"
5,2020-06-03 14:45:00,6,купить квартиру,серебрица.рф,Квартиры в Опалихе от 4 млн руб. – Миниполис Серебрица,"Всего 5 мин. пешком до МЦД-2 ""Опалиха"". С отделкой - от 4,7 млн руб. Звоните! · 5 мин. до МЦД-2 Опалиха. Охраняемая территория. Дворы без машин · Проектная декларация на рекламируемом сайте",серебрица.рф›купить квартиру,Студии от 29 м²,1-комн от 31 м²,2-комн от 50 м²,3-комн от 61 м²,+7 (495) 241-40-64,пн-вс 10:00-21:00
6,2020-06-03 14:45:00,7,купить квартиру,samoletgroup.ru,Купите квартиру – от Группы Самолет!,"Своя квартира с отделкой от 15 000 ₽/мес. Сдача 2 кв. 2020г. Ипотека от 4,5%. Звоните! · Пригород Лесное. Люберцы. Остафьево. Спутник. Квартал Некрасовка. Большое Путилково. Алхимово. Томилино Парк · Проектная декларация на рекламируемом сайте. З… Читать ещёСвоя квартира с отделкой от 15 000 ₽/мес. Сдача 2 кв. 2020г. Ипотека от 4,5%. Звоните! · Пригород Лесное. Люберцы. Остафьево. Спутник. Ква...",samoletgroup.ru›купить квартиру,Акции,Квартира — это не мечта,Ипотека,Выбор квартиры,+7 (495) 153-91-36,пн-вс 9:00-21:00
7,2020-06-03 14:45:00,8,купить квартиру,жкборисоглебское.рф,Купите квартиру в Новой Москве! – ЖК «Борисоглебское»,"Квартиры в Новой Москве от 2,67 млн руб. Специальная скидка до 7%! Успейте купить! · Экологически чистый район. Рядом с Ж/Д станцией. Новая Москва. Малоэтажная застройка. Рядом с МЦД · Проектная деклар

In [7]:
# Посмотрим среднюю позицию домена и какое количесво раз он встретился
pivot = pd.pivot_table(df, index = 'Домен', aggfunc = {'Позиция': 'mean', 'Домен': 'count'})
pivot

,Домен,Позиция
Домен,,
a101.ru,2,1
afi-park.ru,1,9
avito.ru,1,4
egsnk.ru,1,4
etalonnormand.ru,1,2
life.grand-deluxe.ru,1,5
nb2.bicerovo.ru,1,8
novogradpavlyno.ru,1,9
pik.ru,2,6


In [8]:
# Посмотрим есть ли слова "акция" и "скидки" а заголовке или тексте.
# case=False отключает чувствительность к регистру
df.loc[df['Текст'].str.contains("скид|акци", case=False) | df['Заголовок'].str.contains("скид|акци", case=False), ['Текст','Позиция']]

,Текст,Позиция
0,Скидки до 5%! ЖК рядом с метро. С отделкой и без. Прописка в Москве. Звоните сейчас! · Инфраструктура. Ипотека: год без платежей. м. Новомосковская 700м. · Проектная декларация на рекламируемом сайте. З… Читать ещёСкидки до 5%! ЖК рядом с метро. С отделкой и без. Прописка в Москве. Звоните сейчас! · Инфраструктура. Ипотека: год без платежей. м. Новомосковская 700м. · Проектная декларация на ре...,1
7,"Квартиры в Новой Москве от 2,67 млн руб. Специальная скидка до 7%! Успейте купить! · Экологически чистый район. Рядом с Ж/Д станцией. Новая Москва. Малоэтажная застройка. Рядом с МЦД · Проектная декларация на рекламируемом сайте. З… Читать ещёКвартиры в Новой Москве от 2,67 млн руб. Специальная скидка до 7%! Успейте купить! · Экологически чистый район. Рядом с Ж/Д станцией. Новая Москва. Малоэ...",8
15,Закрытая территория. Собственные террасы. Вид на Москва-Сити. Звоните! · 20 мин до Кремля. Детские сады. Двор без машин. Река Сетунь. Потолки до 6м. Частные школы. Матвеевский лес. Коллекционные квартиры,6
17,"Впервые и до 29.02! Успей получить сертификат и купить квартиру по цене от 1,85 млн! · Школа, детский сад. ТЦ на территории. Ипотека по 2 док-ам. Безопасный двор. 17 км до МКАД. Малоэтажные дома. Теплые дома. Бессветофорное шоссе · Проектная декларация на рекламируемом сайте",8


In [ ]:
# Выгрузим результат в csv
df.to_csv('out.csv', sep=',', encoding='utf-8')